##第六章 运营管理##

In [ ]:
# Workforce Scheduling for Anonymous Bank Call Center (Python)

# import packages for analysis and modeling
import pandas as pd  # data frame operations
import numpy as np  # arrays and math functions
import datetime
from rpy2.robjects import r  # interface from Python to R

# Erlang C queueing theory  
# input c = number of servers (positive integer)
#       r = ratio of arrival rate over service rate 
# output = probability of waiting in queue (min 0, max 1)
# adapted from Pedro Canadilla (2014) function 
# C_erlang in the R queueing package 
def erlang_C (c = 1, r = 0):
    if (c <= 0):
        return(1)
    if (r <= 0):
        return(0)
    c = int(c)    
    tot = 1
    for i in range(c-1):
        i = i + 1
        tot = 1 + (tot * i * (1/r))
    return(max(0, min(1, (r * (1/tot)) / (c - (r * (1 - (1/tot)))))))

In [ ]:
# focus upon February 1999
call_center_input_data = pd.read_table('data_anonymous_bank_february.txt')
# examine the structure of these data
print(call_center_input_data.head)

# delete PHANTOM calls
call_center_data = \
    call_center_input_data[call_center_input_data['outcome'] != 'PHANTOM']

# negative VRU times make no sense... drop these rows from data frame
call_center_data = call_center_data[call_center_data['vru_time'] >= 0]

# calculate wait time as sum of vru_time and q_time
call_center_data['wait_time'] = call_center_data['vru_time'] + \
    call_center_data['q_time']

# define date variable with apply and lambda function
call_center_data['date'] = \
    call_center_data['date']\
    .apply(lambda d: datetime.datetime.strptime(str(d), '%y%m%d'))

# define day of week as an integer 0 = Monday 6 = Sunday
call_center_data['day_of_week'] = \
    call_center_data['date'].apply(lambda d: d.weekday())
# use dictionary object for mapping day_of_week to string
day_of_week_to_string = {0 : 'Monday', 
     1 : 'Tuesday', 
     2 : 'Wednesday', 
     3 : 'Thursday', 
     4 : 'Friday',
     5 : 'Saturday',
     6 : 'Sunday'}
call_center_data['day_of_week'] = \
    call_center_data['day_of_week'].map(day_of_week_to_string)
# check structure and contents of the data frame
print(call_center_data.head)

# examine frequency of calls by day of week
print(call_center_data['day_of_week'].value_counts())

<bound method NDFrame.head of       vru+line  call_id  customer_id  ...  ser_exit ser_time     server
0       AA0101    34536          0.0  ...   7:05:41      166      DORIT
1       AA0101    34537          0.0  ...   7:31:59        5  NO_SERVER
2       AA0101    34538          0.0  ...   7:56:21       92       TOVA
3       AA0101    34539   23317894.0  ...   0:00:00        0  NO_SERVER
4       AA0101    34540   48178511.0  ...   8:22:04      104     MICHAL
...        ...      ...          ...  ...       ...      ...        ...
33339   AA0216     5607          0.0  ...   0:00:00        0  NO_SERVER
33340   AA0216     5608          0.0  ...   0:00:00        0  NO_SERVER
33341   AA0216     5609          0.0  ...   0:00:00        0  NO_SERVER
33342   AA0216     5610          0.0  ...  12:22:54      246       YITZ
33343   AA0216     5611          0.0  ...   0:00:00        0  NO_SERVER

[33344 rows x 17 columns]>
<bound method NDFrame.head of       vru+line  call_id  customer_id  ...     se

In [ ]:
# identify the hour of entry into the system
call_center_data['vru_entry'] = \
    call_center_data['vru_entry']\
    .apply(lambda d: datetime.datetime.strptime(str(d), '%H:%M:%S'))
call_center_data['call_hour'] = \
    call_center_data['vru_entry'].apply(lambda d: d.hour)

# check frequency of calls in February by hour and day of week
# note that pandas alphabetizes on output 
print(pd.crosstab(call_center_data['day_of_week'],\
    call_center_data['call_hour'], margins = False))

call_hour    0   1   2   3   4   5   6   ...   17   18   19   20   21   22   23
day_of_week                              ...                                   
Friday       26   2   7   3   5   3  10  ...    4    5    3    8    3    0    1
Monday       13   8   1   1   0   4  26  ...  262  363  258  250  208  190  191
Saturday      0   2   0   0   0   0   0  ...    6   21  237  165  145  124  136
Sunday       20   7   2   1   0   3  14  ...  494  302  244  169  364  259  133
Thursday     25   2   2   0   0   5  21  ...  402  295  213  212  162  174  145
Tuesday      11   6   4   2   2   6  21  ...  429  353  275  271  259  211  167
Wednesday    27   7   5   0   2   1  19  ...  300  355  342  272  246  230  177

[7 rows x 24 columns]


In [ ]:
# create an ordered table for Frequency of calls
table_data = call_center_data.loc[:,['day_of_week', 'call_hour']]
day_of_week_to_ordered_day_of_week = {'Monday' : '2_Monday', 
     'Tuesday' : '3_Tuesday', 
     'Wednesday' : '4_Wednesday', 
     'Thursday' : '5_Thursday', 
     'Friday' : '6_Friday',
     'Saturday' : '7_Saturday',
     'Sunday' : '1_Sunday'}
table_data['ordered_day_of_week'] = \
    table_data['day_of_week'].map(day_of_week_to_ordered_day_of_week)
print(pd.crosstab(table_data['ordered_day_of_week'],\
    table_data['call_hour'], margins = False))

call_hour            0   1   2   3   4   5   ...   18   19   20   21   22   23
ordered_day_of_week                          ...                              
1_Sunday             20   7   2   1   0   3  ...  302  244  169  364  259  133
2_Monday             13   8   1   1   0   4  ...  363  258  250  208  190  191
3_Tuesday            11   6   4   2   2   6  ...  353  275  271  259  211  167
4_Wednesday          27   7   5   0   2   1  ...  355  342  272  246  230  177
5_Thursday           25   2   2   0   0   5  ...  295  213  212  162  174  145
6_Friday             26   2   7   3   5   3  ...    5    3    8    3    0    1
7_Saturday            0   2   0   0   0   0  ...   21  237  165  145  124  136

[7 rows x 24 columns]


In [ ]:
# select first week of February 1999 for data visualization and analysis
# that week began on Monday, February 1 and ended on Sunday, February 7        
selected_week = call_center_data[call_center_data['date'] < 
    np.datetime64(datetime.date(1999, 2, 8))]
print(selected_week.head)  

<bound method NDFrame.head of       vru+line  call_id  customer_id  ...  wait_time day_of_week call_hour
0       AA0101    34536          0.0  ...          9      Monday         7
1       AA0101    34537          0.0  ...         10      Monday         7
2       AA0101    34538          0.0  ...         13      Monday         7
3       AA0101    34539   23317894.0  ...         11      Monday         8
4       AA0101    34540   48178511.0  ...         39      Monday         8
...        ...      ...          ...  ...        ...         ...       ...
33236   AA0216     5504          0.0  ...         12      Sunday        11
33237   AA0216     5505          0.0  ...         90      Sunday        13
33238   AA0216     5506          0.0  ...         13      Sunday        14
33239   AA0216     5507          0.0  ...         20      Sunday        20
33240   AA0216     5508          0.0  ...         15      Sunday        21

[8940 rows x 20 columns]>


In [ ]:
# check frequency of calls in February by hour and day of week
# create an ordered table for frequency of calls 
# for the first week in February 1999
table_data = selected_week.loc[:,['day_of_week', 'call_hour']]
day_of_week_to_ordered_day_of_week = {'Monday' : '2_Monday', 
     'Tuesday' : '3_Tuesday', 
     'Wednesday' : '4_Wednesday', 
     'Thursday' : '5_Thursday', 
     'Friday' : '6_Friday',
     'Saturday' : '7_Saturday',
     'Sunday' : '1_Sunday'}
table_data['ordered_day_of_week'] = \
    table_data['day_of_week'].map(day_of_week_to_ordered_day_of_week)
print(pd.crosstab(table_data['ordered_day_of_week'],\
    table_data['call_hour'], margins = False))

call_hour            0   1   2   3   4   5   6   ...   17   18   19  20  21  22  23
ordered_day_of_week                              ...                               
1_Sunday              7   2   1   1   0   2   6  ...  103   64   43  50  62  47  42
2_Monday              4   3   1   0   0   1   9  ...   68  120   69  87  61  49  45
3_Tuesday             2   2   1   0   0   4   9  ...  115   81   89  66  75  51  55
4_Wednesday           7   2   2   0   1   1   5  ...   68   83  104  66  68  49  46
5_Thursday            6   0   0   0   0   1   2  ...  106   81   70  62  34  43  36
6_Friday              3   0   4   1   1   1   2  ...    0    0    2   1   0   0   0
7_Saturday            0   2   0   0   0   0   0  ...    0    5   66  37  37  27  28

[7 rows x 24 columns]


In [ ]:
# wait-time ribbons were created with R ggplot2 software
# Python packages ggplot or rpy2 could be used for plotting

# select Wednesdays in February for the queueing model
wednesdays = call_center_data[call_center_data['day_of_week'] == \
    'Wednesday'] 
print(wednesdays.head) 

<bound method NDFrame.head of       vru+line  call_id  customer_id  ...  wait_time day_of_week call_hour
158     AA0101    34698          0.0  ...         38   Wednesday         6
159     AA0101    34699   28626448.0  ...         29   Wednesday         7
160     AA0101    34700          0.0  ...         12   Wednesday         8
161     AA0101    34701          0.0  ...         11   Wednesday         8
162     AA0101    34702          0.0  ...         14   Wednesday         8
...        ...      ...          ...  ...        ...         ...       ...
33325   AA0216     5593          0.0  ...         12   Wednesday        12
33326   AA0216     5594          0.0  ...         90   Wednesday        13
33327   AA0216     5595          0.0  ...         14   Wednesday        15
33328   AA0216     5596          0.0  ...        300   Wednesday        18
33329   AA0216     5597          0.0  ...         77   Wednesday        19

[6289 rows x 20 columns]>


In [ ]:
# arrival rate as average number of calls into VRU per hour 
arrived_for_hour = wednesdays['call_hour'].value_counts()
check_hourly_arrival_rate = arrived_for_hour/4  # four Wednesdays in February 1999
print(check_hourly_arrival_rate)

13    203.50
10    124.00
14    115.75
15    115.50
11    110.25
9     107.50
8      97.25
12     95.50
18     88.75
19     85.50
17     75.00
20     68.00
16     67.75
21     61.50
22     57.50
23     44.25
7      39.50
0       6.75
6       4.75
1       1.75
2       1.25
4       0.50
5       0.25
Name: call_hour, dtype: float64


In [ ]:
# organize hourly arrival rates according to 24-hour clock
hourly_arrival_rate = [6.75, 1.75, 1.25, 0.00, 0.50, 0.25,\
    4.75, 39.50, 97.25,107.50, 124.00,110.25, 95.50,\
    203.50, 115.75, 115.50, 67.75, 75.00, 88.75,\
    85.50, 68.00, 61.50, 57.50, 44.25]

# service times may vary hour-by-hour due to differences 
# in service requests and individuals calling hour-by-hour
# begin by selecting calls that receive service
wednesdays_served = wednesdays[wednesdays['server'] != \
    'NO_SERVER'] 
print(wednesdays_served.head) 

<bound method NDFrame.head of       vru+line  call_id  customer_id  ...  wait_time day_of_week call_hour
159     AA0101    34699   28626448.0  ...         29   Wednesday         7
161     AA0101    34701          0.0  ...         11   Wednesday         8
162     AA0101    34702          0.0  ...         14   Wednesday         8
163     AA0101    34703          0.0  ...        204   Wednesday         8
164     AA0101    34704   59778696.0  ...         33   Wednesday         8
...        ...      ...          ...  ...        ...         ...       ...
33323   AA0216     5591          0.0  ...        155   Wednesday        11
33324   AA0216     5592          0.0  ...         15   Wednesday        12
33325   AA0216     5593          0.0  ...         12   Wednesday        12
33327   AA0216     5595          0.0  ...         14   Wednesday        15
33328   AA0216     5596          0.0  ...        300   Wednesday        18

[4719 rows x 20 columns]>


In [ ]:
hourly_mean_service_time =\
    wednesdays_served.pivot_table('ser_time', columns = ['call_hour'],\
    aggfunc = 'mean', margins = False)

# hourly service rate given the current numbers of service operators
served_for_hour = wednesdays_served['call_hour'].value_counts()
print(served_for_hour)

15    400
10    393
14    384
9     339
11    333
13    327
8     290
19    290
12    288
18    284
17    254
16    250
20    238
21    204
22    170
23    144
7     128
6       3
Name: call_hour, dtype: int64


In [ ]:
# compute service rate noting that there are 3600 seconds in an hour
# adding 60 seconds to each mean service time for time between calls
# this 60 seconds is the wrap up time or time an service agent remains 
# unavailable to answer a new call after a call has been completed
mean_hourly_service_rate = 3600/(hourly_mean_service_time.mean() + 60)
print('\nHourly Service Rate for Wednesdays:',\
    round(mean_hourly_service_rate,3))


Hourly Service Rate for Wednesdays: call_hour
6      8.948
7     17.951
8     15.668
9     13.239
10    14.252
11    13.185
12    12.135
13    14.290
14    14.828
15    15.000
16    16.136
17    16.754
18    14.973
19    15.353
20    15.415
21    15.425
22    16.783
23    17.666
dtype: float64


In [ ]:
# use 15 calls per hour as the rate for one service operator
SERVICE_RATE = 15

# use a target for the probability of waiting in queue to be 0.50
PROBABILITY_GOAL = 0.50

# Erlang C queueing calculations with Python erlang_C function
# inputs c = number of servers
#        r = ratio of rate of arrivals and rate of service
# returns the propability of waiting in queue because all servers are busy
# use while-loop iteration to determine the number of servers needed 
# we do this for each hour of the day knowing the hourly arrival rate
servers_needed = [0] * 24
for index_for_hour in range(24):
    if (hourly_arrival_rate[index_for_hour] > 0):
        erlang_probability = 1 # initialize on entering while-loop
        while (erlang_probability > PROBABILITY_GOAL):
            servers_needed[index_for_hour] = servers_needed[index_for_hour] + 1
            erlang_probability = \
                erlang_C(c = servers_needed[index_for_hour],\
                    r = hourly_arrival_rate[index_for_hour]/SERVICE_RATE)
print(servers_needed)  # check queueing theory result 

[1, 1, 1, 0, 1, 1, 1, 4, 8, 9, 10, 9, 8, 16, 10, 10, 6, 7, 8, 8, 6, 6, 5, 4]


In [ ]:
# the result for servers.needed is obtained as
# 1  1  1  0  1  1  1  4  8  9 10  9  8 16 10 10  6  7  8  8  6  6  5  4
# we will assume the bank call center will be closed hours 00 through 05
# but use the other values as the bank's needed numbers of servers
for index_for_hour in range(6):
    servers_needed[index_for_hour] = 0
print('\nHourly Operator Requirements:\n',servers_needed)


Hourly Operator Requirements:
 [0, 0, 0, 0, 0, 0, 1, 4, 8, 9, 10, 9, 8, 16, 10, 10, 6, 7, 8, 8, 6, 6, 5, 4]


In [ ]:
# read in case data for the structure of call center worker shifts
bank_shifts_data_frame = pd.read_csv("data_anonymous_bank_shifts.csv")
# examine the structure of these data
print(bank_shifts_data_frame.head)

<bound method NDFrame.head of     Hour StartTime  Shift1  Shift2  ...  Shift5  Shift6  Shift7  Shift8
0      1  Midnight       1       0  ...       0       0       0       0
1      2       1am       1       0  ...       0       0       0       0
2      3       2am       1       0  ...       0       0       0       0
3      4       3am       1       0  ...       0       0       0       0
4      5       4am       1       0  ...       0       0       0       0
5      6       5am       1       0  ...       0       0       0       0
6      7       6am       0       1  ...       0       0       0       0
7      8       7am       0       1  ...       0       0       0       0
8      9       8am       0       1  ...       0       0       0       0
9     10       9am       0       1  ...       0       0       0       0
10    11      10am       0       1  ...       0       0       0       0
11    12      11am       0       1  ...       0       0       0       0
12    13      Noon       0       0

In [ ]:
# constraint matrix as required for mathematical programming
constraint_matrix = np.array(bank_shifts_data_frame)[:,2:]
# we will create this type of object on the R side as well

# six-hour shift salaries in Israeli sheqels 
# 1 ILS = 3.61 USD in June 2013
# these go into the objective function for integer programing
# with the objective of minimizing total costs
cost_vector = [252, 288, 180, 180, 180, 288, 288, 288] 

# install lpsolove package and drivers for Python 
# noting the operating system being used
# or use rpy2 access to lpSolve in R as shown here

# assign lists from Python to R using rpy2
r.assign('servers_needed_R', servers_needed)
r.assign('cost_vector_R', cost_vector)

r('bank.shifts.data.frame <- read.csv("data_anonymous_bank_shifts.csv")')
r('constraint_matrix_R <- as.matrix(bank.shifts.data.frame[,3:10])')

# check mathematical programming inputs on the R side
r('print(as.numeric(unlist(servers_needed_R)))')
r('print(as.numeric(unlist(cost_vector_R)))')
r('print(constraint_matrix_R)')

 [1]  0  0  0  0  0  0  1  4  8  9 10  9  8 16 10 10  6  7  8  8  6  6  5  4
[1] 252 288 180 180 180 288 288 288
      Shift1 Shift2 Shift3 Shift4 Shift5 Shift6 Shift7 Shift8
 [1,]      1      0      0      0      0      0      0      0
 [2,]      1      0      0      0      0      0      0      0
 [3,]      1      0      0      0      0      0      0      0
 [4,]      1      0      0      0      0      0      0      0
 [5,]      1      0      0      0      0      0      0      0
 [6,]      1      0      0      0      0      0      0      0
 [7,]      0      1      0      0      0      0      0      0
 [8,]      0      1      0      0      0      0      0      0
 [9,]      0      1      1      0      0      0      0      0
[10,]      0      1      1      0      0      0      0      0
[11,]      0      1      1      1      0      0      0      0
[12,]      0      1      1      1      0      0      0      0
[13,]      0      0      1      1      1      0      0      0
[14,]      0      0

1,1,1,...,1,1,1


In [ ]:
# solve the mathematical programming problem
r('library(lpSolve)')  
r('call_center_schedule <- lp(const.mat=constraint_matrix_R,\
    const.rhs = as.numeric(unlist(servers_needed_R)),\
    const.dir = rep(">=", times = 8),\
    int.vec = 1:8,\
    objective = as.numeric(unlist(cost_vector_R)),\
    direction = "min")')
    
# prepare summary of the results for the call center problem
# working on the R side
r('ShiftID <- 1:8')
r('StartTime <- c(0,6,8,10,12,2,4,6)')
# c("Midnight","6 AM","8 AM","10 AM","Noon","2 PM","4 PM","6 PM")
r('ShiftDuration <- rep(6,times=8)')
r('HourlyShiftSalary <- c(42,48,30,30,30,48,48,48)')
r('HourlyShiftCost <- call_center_schedule$objective') # six x hourly shift salary
r('Solution <- call_center_schedule$solution')  
r('ShiftCost <- call_center_schedule$solution * call_center_schedule$objective')
r('call_center_summary <- \
  data.frame(ShiftID,StartTime,ShiftDuration,HourlyShiftSalary,\
  HourlyShiftCost,Solution,ShiftCost)')
r('cat("\n\n","Call Center Summary","\n\n")')
r('print(call_center_summary)')
r('print(call_center_schedule)')

# alternatively... bring the solution from R to Python
# and print the minimum-cost solution on the Python side
call_center_schedule = r('call_center_schedule')
print(call_center_schedule)

R[write to console]: Error in library(lpSolve) : there is no package called ‘lpSolve’
Calls: <Anonymous> -> <Anonymous> -> library



RRuntimeError: ignored

In [ ]:
# Suggestion for the student:
# Attack the problem using discrete event simulation, 
# perhaps drawing on the SimPy package.
# Try running a sensitivity test, varying the workforce requirements
# and noting the effect upon the optimal assignment of workers to shifts.
# This can be done in a Python for-loop.